In [1]:
import nltk
from nltk.corpus import stopwords
import csv
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
from indexing import IndexType, BasicInvertedIndex, InvertedIndex
from rank_bm25 import BM25Okapi
import ranker
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
import ast


2023-12-11 20:33:35.679707: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 20:33:35.679754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 20:33:35.681049: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 20:33:35.687951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 20:33:36.919843: W tensorflow/comp

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/yyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/yyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data Preprocessing

In [14]:
# !pip install rank_bm25

In [15]:
# data_sets = {}
# with open('/content/drive/MyDrive/Health.txt', 'r') as file:
#     current_set = None
#     for line in file:
#         line = line.strip()
#         if not line:
#             current_set = None
#             continue
#         parts = line.split(':')
#         current_set = parts[0]
#         if current_set not in data_sets:
#             data_sets[current_set] = []
#         data_sets[current_set].append(parts[1])
# transposed_data = list(map(list, zip(*data_sets.values())))

# with open('output.csv', 'w', newline='') as csvfile:
#     csv_writer = csv.writer(csvfile)
#     csv_writer.writerow(data_sets.keys())
#     csv_writer.writerows(transposed_data)

In [25]:
df = pd.read_csv('/home/yyxia/si650_project/output_health.csv')

In [26]:
df['time'] = df['review/time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y'))
time_counts = df['time'].value_counts()

print("Time Counts:")
print(time_counts)

Time Counts:
2012    110399
2011     73180
2010     50929
2013     42320
2009     39277
2008     37797
2007     34970
2006     14463
2005     10034
2004      5872
2003      3895
2000      2988
2002      1670
2001       731
1999       245
1969        10
1998         1
Name: time, dtype: int64


## Based on the data above, I chose to filter those data from 2005 to 2013.

In [ ]:
df['review/time'] = pd.to_numeric(df['review/time'])
start_timestamp = datetime(2005, 1, 1).timestamp()
end_timestamp = datetime(2013, 12, 31).timestamp()
time_filtered_df = df[(df['review/time'] >= start_timestamp) & (df['review/time'] <= end_timestamp)]

In [ ]:
score_filtered_df = time_filtered_df[time_filtered_df['review/score'] > 3]

In [ ]:
helpfulness_filtered_df = score_filtered_df[~score_filtered_df['review/helpfulness'].str.startswith(' 0/')]

In [ ]:
category_counts = helpfulness_filtered_df['product/title'].value_counts()

print("Category Counts:")
print(category_counts)

filtered_categories = category_counts[category_counts >= 10].index

filtered_df = helpfulness_filtered_df[helpfulness_filtered_df['product/title'].isin(filtered_categories)]

# Display the filtered DataFrame
print("\nFiltered DataFrame:")
print(filtered_df.shape)

In [ ]:
filtered_df.head(5)

## Now we get 2834 different products in general.

In [ ]:
print(filtered_df['product/title'].describe())

In [ ]:
print(df['product/title'].describe())

# Tokenizing Data

In [27]:
tokenizer = RegexpTokenizer(r'\w+')
def preprocess_dataset(index, column, dataset):
    text = dataset[column][index]
    tokens = tokenizer.tokenize(text)
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in set(stopwords.words('english'))]
    dataset[column][index] = filtered_tokens

In [28]:
filtered_df.head()

,product/productId,product/title,product/price,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text,time,docid,review_text_list
0,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,ASL7LZAL2LBMS,"Ohio Nurse ""bronwyn59""",13/13,5.0,1157241600,Lower leg comfort for 12 hours +,"['excellent', 'stockings', 'long', 'shifts', '...",2006,1,"[excellent, stockings, long, shifts, feet, tig..."
1,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A21F8C1MDLQDTH,S. Lawson,14/15,5.0,1110326400,Fashionable Compression Stockings!,"['dvt', 'doctor', 'required', 'wear', 'compres...",2005,2,"[dvt, doctor, required, wear, compression, sto..."
2,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A3J4K29TNC7RV1,"Karen W. ""Karen W.""",13/14,5.0,1173312000,Sheer Relief!,"['doctor', 'recommended', 'thigh', 'high', 've...",2007,3,"[doctor, recommended, thigh, high, version, pr..."
3,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A3GQAVM66FA88J,"G. Alosa ""wifey""",6/6,5.0,1204416000,No more uncomfortable flights,"['stockings', 'fit', 'well', 'comfortable', 'o...",2008,4,"[stockings, fit, well, comfortable, opportunit..."
4,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A1BTRWL1E3T2YB,A. S. Penuel,6/6,5.0,1184803200,Jobst UltraSheer Thigh High,"['excellent', 'product', 'however', 'difficult...",2007,5,"[excellent, product, however, difficult, get, ..."


In [29]:
filtered_df['titleNsum'] = filtered_df['product/title'] + ' ' + filtered_df['review/summary']

In [32]:
filtered_df.columns

Index(['product/productId', 'product/title', 'product/price', 'review/userId',
       'review/profileName', 'review/helpfulness', 'review/score',
       'review/time', 'review/summary', 'review/text', 'time', 'docid',
       'review_text_list', 'titleNsum'],
      dtype='object')

In [33]:
filtered_df.head()

,product/productId,product/title,product/price,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text,time,docid,review_text_list,titleNsum
0,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,ASL7LZAL2LBMS,"Ohio Nurse ""bronwyn59""",13/13,5.0,1157241600,Lower leg comfort for 12 hours +,"['excellent', 'stockings', 'long', 'shifts', '...",2006,1,"[excellent, stockings, long, shifts, feet, tig...",Jobst Ultrasheer 15-20 Knee-High Silky Beige ...
1,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A21F8C1MDLQDTH,S. Lawson,14/15,5.0,1110326400,Fashionable Compression Stockings!,"['dvt', 'doctor', 'required', 'wear', 'compres...",2005,2,"[dvt, doctor, required, wear, compression, sto...",Jobst Ultrasheer 15-20 Knee-High Silky Beige ...
2,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A3J4K29TNC7RV1,"Karen W. ""Karen W.""",13/14,5.0,1173312000,Sheer Relief!,"['doctor', 'recommended', 'thigh', 'high', 've...",2007,3,"[doctor, recommended, thigh, high, version, pr...",Jobst Ultrasheer 15-20 Knee-High Silky Beige ...
3,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A3GQAVM66FA88J,"G. Alosa ""wifey""",6/6,5.0,1204416000,No more uncomfortable flights,"['stockings', 'fit', 'well', 'comfortable', 'o...",2008,4,"[stockings, fit, well, comfortable, opportunit...",Jobst Ultrasheer 15-20 Knee-High Silky Beige ...
4,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A1BTRWL1E3T2YB,A. S. Penuel,6/6,5.0,1184803200,Jobst UltraSheer Thigh High,"['excellent', 'product', 'however', 'difficult...",2007,5,"[excellent, product, however, difficult, get, ...",Jobst Ultrasheer 15-20 Knee-High Silky Beige ...


In [32]:
for index, row in tqdm(filtered_df.iterrows()):
    preprocess_dataset(index, 'review/text', filtered_df)

0it [00:00, ?it/s]/tmp/ipykernel_1939527/1026307363.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column][index] = filtered_tokens
106654it [41:49, 42.51it/s]


In [34]:
for index, row in tqdm(filtered_df.iterrows()):
    preprocess_dataset(index, 'titleNsum', filtered_df)

0it [00:00, ?it/s]/tmp/ipykernel_1079874/1026307363.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column][index] = filtered_tokens
106654it [07:15, 245.17it/s]


In [40]:
filtered_df.head()

,product/productId,product/title,product/price,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text,time,docid,review_text_list,titleNsum
0,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,ASL7LZAL2LBMS,"Ohio Nurse ""bronwyn59""",13/13,5.0,1157241600,Lower leg comfort for 12 hours +,"['excellent', 'stockings', 'long', 'shifts', '...",2006,1,"[excellent, stockings, long, shifts, feet, tig...","[jobst, ultrasheer, 15, 20, knee, high, silky,..."
1,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A21F8C1MDLQDTH,S. Lawson,14/15,5.0,1110326400,Fashionable Compression Stockings!,"['dvt', 'doctor', 'required', 'wear', 'compres...",2005,2,"[dvt, doctor, required, wear, compression, sto...","[jobst, ultrasheer, 15, 20, knee, high, silky,..."
2,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A3J4K29TNC7RV1,"Karen W. ""Karen W.""",13/14,5.0,1173312000,Sheer Relief!,"['doctor', 'recommended', 'thigh', 'high', 've...",2007,3,"[doctor, recommended, thigh, high, version, pr...","[jobst, ultrasheer, 15, 20, knee, high, silky,..."
3,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A3GQAVM66FA88J,"G. Alosa ""wifey""",6/6,5.0,1204416000,No more uncomfortable flights,"['stockings', 'fit', 'well', 'comfortable', 'o...",2008,4,"[stockings, fit, well, comfortable, opportunit...","[jobst, ultrasheer, 15, 20, knee, high, silky,..."
4,B000278ADA,Jobst Ultrasheer 15-20 Knee-High Silky Beige ...,46.34,A1BTRWL1E3T2YB,A. S. Penuel,6/6,5.0,1184803200,Jobst UltraSheer Thigh High,"['excellent', 'product', 'however', 'difficult...",2007,5,"[excellent, product, however, difficult, get, ...","[jobst, ultrasheer, 15, 20, knee, high, silky,..."


In [43]:
filtered_df.to_csv('output_tokenized.csv', index=False)